In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [8]:
data = pd.read_csv("monte_carlo_200.csv")

In [9]:
class NashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        
        #calculate the utility from each action
        u1 = np.dot(self.exog[:,0:k], params[0:k])  + params[k]*self.endog[:,1]
        u2 = np.dot(self.exog[:,k:], params[k+1:2*k+1]) + params[2*k+1]*self.endog[:,0]
        
        #calculate the log-likelihood
        ll = np.exp(u1)**self.endog[:,0]/( 1.+np.exp(u1) )
        ll = ll*np.exp(u2)**self.endog[:,1]/( 1.+np.exp(u2) )
        
        #adjust for eq selection
        mult01 = self.endog[:,1]*(1-self.endog[:,0])
        mult = self.endog[:,0]*(1-self.endog[:,1]) + self.endog[:,1]*(1-self.endog[:,0])
        ll = ll* np.exp( params[2*k+2] )**mult01 /( 1+ np.exp(params[2*k+2]) )**mult
        return -np.log(ll).sum()
    
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+3)
        #start_params = np.array([2,3,-2,2,3,-2,.2])
        return super(NashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = data.shape[0]
y = data[['y1','y2']]
x = data[['x1','x2']]

x.insert(0,'x01',np.ones(N))
x.insert(2,'x02',np.ones(N))

model_200 = NashLogit(y,x)
model_200 = model_200.fit()
print(model_200.summary(xname=["b01","b11","d1","b02","b12","d2","rho"]))

                              NashLogit Results                               
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -221.85
Model:                      NashLogit   AIC:                             449.7
Method:            Maximum Likelihood   BIC:                             459.6
Date:                Fri, 03 Apr 2020                                         
Time:                        12:25:02                                         
No. Observations:                 200                                         
Df Residuals:                     197                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
b01            1.2742      0.387      3.289      0.001       0.515       2.034
b11            3.2402      0.536      6.047      0.0

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [10]:
data = pd.read_csv("monte_carlo_1000.csv")
N = data.shape[0]
y = data[['y1','y2']]
x = data[['x1','x2']]

x.insert(0,'x01',np.ones(N))
x.insert(2,'x02',np.ones(N))

model_1000 = NashLogit(y,x)
model_1000 = model_1000.fit()
print(model_1000.summary(xname=["b01","b11","d1","b02","b12","d2","rho"]))

                              NashLogit Results                               
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -987.73
Model:                      NashLogit   AIC:                             1981.
Method:            Maximum Likelihood   BIC:                             1996.
Date:                Fri, 03 Apr 2020                                         
Time:                        12:25:09                                         
No. Observations:                1000                                         
Df Residuals:                     997                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
b01            1.6448      0.198      8.310      0.000       1.257       2.033
b11            4.2165      0.297     14.210      0.0

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
